In [ ]:
!pip install -r requirements.txt

In [ ]:
import asyncio
import logging

from azure.ai.projects.aio import AIProjectClient
from azure.ai.projects.models import AzureAISearchTool, ConnectionType
from azure.identity.aio import DefaultAzureCredential

from semantic_kernel.agents.azure_ai import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole

logging.basicConfig(level=logging.WARNING)

In [3]:
import asyncio
import os
from dotenv import load_dotenv
from azure.ai.projects.models import AzureAISearchTool
from azure.identity.aio import AzureCliCredential
from semantic_kernel.agents.azure_ai import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole

# Load environment variables
load_dotenv()

async def search_azure_docs(question: str):
    # Get configuration from environment variables
    search_connection_name = os.getenv("AI_SEARCH_CONNECTION_NAME", "fsunavalaaisearch")
    index_name = os.getenv("AI_SEARCH_INDEX_NAME", "default")
    
    # Set up agent settings
    settings = AzureAIAgentSettings.create()
    
    async with AzureCliCredential() as creds, AzureAIAgent.create_client(
        credential=creds,
        conn_str=settings.project_connection_string.get_secret_value(),
    ) as client:
        # Find search connection by name
        connections = await client.connections.list()
        search_conn = next((c.id for c in connections if getattr(c, 'name', '') == search_connection_name), None)
        
        if not search_conn:
            raise ValueError(f"Connection '{search_connection_name}' not found")

        # Create search tool and agent
        search_tool = AzureAISearchTool(
            index_connection_id=search_conn,
            index_name=index_name
        )

        agent = AzureAIAgent(
            client=client,
            definition=await client.agents.create_agent(
                model=settings.model_deployment_name,
                instructions="You are an Azure AI Search expert. Use the Azure AI Search Tool to provide accurate, concise answers about Azure AI Search documentation.",
                tools=search_tool.definitions,
                tool_resources=search_tool.resources,
                headers={"x-ms-enable-preview": "true"}
            )
        )

        # Set up conversation
        thread = await client.agents.create_thread()
        await agent.add_chat_message(
            thread_id=thread.id,
            message=ChatMessageContent(role=AuthorRole.USER, content=question)
        )

        # Process query and cleanup
        try:
            async for response in agent.invoke(thread_id=thread.id):
                if response.role != AuthorRole.TOOL:
                    print(f"🤖 {response.content}")
        finally:
            # Clean up resources
            await client.agents.delete_thread(thread.id)
            await client.agents.delete_agent(agent.id)

if __name__ == "__main__":
    # For Jupyter notebooks
    import nest_asyncio
    nest_asyncio.apply()
    
    print("🔍 Searching Azure AI Search docs...")
    asyncio.run(search_azure_docs("What is BQ in Azure AI Search"))

🔍 Searching Azure docs...
🤖 In Azure AI Search, "BQ" refers to Binary Quantization. It is a technique used to compress high-dimensional vectors by representing each component as a single bit (0 or 1), significantly reducing the memory footprint and speeding up vector comparison operations. This method is particularly effective for embeddings that are centered around zero and with dimensions greater than 1024. By utilizing BQ, the size of vector indices can be reduced by up to 96%, which enhances performance in search and retrieval tasks【3:0†source】.
